# Data cleaning and data wrangling on Availability Calendar database
## 1. Loading data

In [1]:
# import pandas
import pandas as pd


In [2]:
# read data from a zipped file to a dataframe
df = pd.read_csv('calendar.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False, low_memory=False)

In [3]:
# check data frame info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22200690 entries, 0 to 22200689
Data columns (total 7 columns):
listing_id        int64
date              object
available         object
price             object
adjusted_price    object
minimum_nights    float64
maximum_nights    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.2+ GB


### 2. Dealing with missing values


In [4]:
# check quantity of missing values
print('There are '+str(len(df)-len(df.dropna()))+' of rows with at least 1 missing value.')

There are 205 of rows with at least 1 missing value.


There are only 205 rows with missing data in 22MM large data set. Therefore these rows can be dropped.


In [5]:
# drop missing values
df = df.dropna()

In [6]:
df.isnull().values.any()

False

### 3. Data Transformation
I will change price columns from the string to the float and date column to datetime.

In [7]:
# define function that transforms column with string prices to numeric prices
def str_to_num(col_name, df):
    """Return a new column with numeric prices that correspond given string values in specified column"""
    new_col = df[col_name].str.strip('$')
    new_col = new_col.str.replace(',','')
    new_col = pd.to_numeric(new_col)
    return new_col

In [8]:
# apply function to both price columns
col_names = ['price', 'adjusted_price']
for col in col_names:
    df[col] = str_to_num(col,df)

In [9]:
# transform date column to datetime type
df['date'] = pd.to_datetime(df.date)

Lastly, I will drop the listings that were dropped from general listing database to keep information consistent.

In [10]:
# load listing ids list
ids = pd.read_csv('listing_ids.csv', header = None)
ids = ids.drop(0, axis=1)

In [11]:
# merge data frames
calendar_cleaned = ids.merge(df, left_on=1, right_on='listing_id')
calendar_cleaned = calendar_cleaned.drop(1, axis = 1)

In [12]:
# write data to file
calendar_cleaned.to_csv('calendar_cleaned.csv.gz', compression='gzip')